# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [20]:
## The following libraries might be useful

#!pip install -q langchain-openai
#!pip install -U -q langchain-community
#!pip install -U -q langchain-chroma
#!pip install -U -q datasets
#!pip install -U -q ragas
#!pip install -U -q rouge_score

!pip install -q langchain-openai
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score

!pip install -U evaluate

   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.1 kB ? eta -:--:--
   ------------------- -------------------- 41.0/84.1 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 84.1/84.1 kB 785.7 kB/s eta 0:00:00


In [21]:
# Import essential libraries

import os
import re
import pandas as pd
import numpy as np

# LangChain imports
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# For TF-IDF analysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK for text preprocessing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# tqdm for progress bars
from tqdm import tqdm

# For OpenAI embeddings with LangChain
from langchain_openai import OpenAIEmbeddings

# For ChromaDB to load embeddings to vector database
from langchain_community.vectorstores import Chroma

# For initialising LLM
from langchain_openai import ChatOpenAI

# Building RAG chain using RetrievalQA
from langchain.chains import RetrievalQA

# For extraction
import json
import glob

# For scoring answers
from rouge_score import rouge_scorer
from evaluate import load as load_metric

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shayakm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [3]:
# Alright, taking LangChain approach for this one

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

base_path = 'corpus'

# Creating empty list to hold all the docs
all_documents = []

# Run through folders
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            try:
                loader = TextLoader(file_path, encoding='utf-8')
                docs = loader.load()
                all_documents.extend(docs)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

# Check how many files
print(f"Total documents loaded: {len(all_documents)}")

Total documents loaded: 695


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [4]:
# Alright, time to clean and preprocess the data
# For this we will create a clean_text function for efficient operations

stop_words = set(stopwords.words('english'))

def clean_text(text):
    try:
        # lowercase conversion
        text = text.lower()

        # remove emails
        text = re.sub(r'\S+@\S+', '', text)

        # remove phone numbers 
        text = re.sub(r'\b\d{10,}\b', '', text)  # long digit strings
        text = re.sub(r'\+?\d{1,3}[-.\s]?\(?\d{1,5}\)?[-.\s]?\d{3,5}[-.\s]?\d{3,5}', '', text)

        # remove special characters (except spaces and periods)
        text = re.sub(r'[^a-zA-Z0-9\s.]', ' ', text)

        # remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        # remove stopwords
        words = text.split()
        words = [word for word in words if word not in stop_words]
        text = ' '.join(words)

        return text
    except Exception as e:
        print(f"Error during cleaning: {e}")
        return None  

#Now, let's apply the function to all docs

cleaned_documents = []

for doc in all_documents:
    cleaned_text = clean_text(doc.page_content)
    if cleaned_text:  
        doc.page_content = cleaned_text
        cleaned_documents.append(doc)

print(f"Documents after cleaning: {len(cleaned_documents)}")

Documents after cleaning: 695


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [5]:
# Calculate the average, maximum and minimum document length.

document_lengths = [len(doc.page_content.split()) for doc in cleaned_documents]

average_length = sum(document_lengths) / len(document_lengths)
max_length = max(document_lengths)
min_length = min(document_lengths)

# Let's see the results
print(f"Average document length: {average_length:.2f} words")
print(f"Maximum document length: {max_length} words")
print(f"Minimum document length: {min_length} words")

Average document length: 9167.10 words
Maximum document length: 86470 words
Minimum document length: 149 words


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [6]:
# Find frequency of occurence of words

from collections import Counter

# combining all document text into one big string
all_text = ' '.join([doc.page_content for doc in cleaned_documents])

# tokenising into words
words = all_text.split()

# filtering out stopwords once more for check
filtered_words = [word for word in words if word not in stop_words and word.isalpha()]

# counting word frequencies
word_counts = Counter(filtered_words)

# the 20 most common words
most_common = word_counts.most_common(20)
print("Top 20 most common words:")
for word, count in most_common:
    print(f"{word}: {count}")

# the 20 least common words (words that occur only once)
least_common = [word for word, count in word_counts.items() if count == 1]
print("\n20 least common words (occur only once):")
for word in least_common[:20]:
    print(word)


Top 20 most common words:
company: 154028
shall: 107747
agreement: 92607
section: 75054
parent: 59791
party: 51424
date: 37509
material: 34109
time: 33500
merger: 32577
b: 32344
subsidiaries: 31766
applicable: 30770
including: 29221
respect: 28669
may: 28050
stock: 25906
information: 24358
prior: 23315
business: 23004

20 least common words (occur only once):
monnaie
candidacy
grateful
sohet
koen
gerven
cvba
peratieve
vennootschap
beperkte
aansprakelijkheid
rative
rpr
brussel
bruxelles
kredbebb
tohmatsu
maidenhead
klingenweg
temasek


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [7]:
# Transform the page contents of documents

# getting cleaned text of all documents
documents_text = [doc.page_content for doc in cleaned_documents]

# now, initialising TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# now, transforming documents into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(documents_text)

print("TF-IDF matrix shape:", tfidf_matrix.shape)


# Compute similarity scores

# computing pairwise cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix[:10])

# now, displaying the similarity matrix

similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

TF-IDF matrix shape: (695, 34409)


0         1         2         3         4         5         6  \
0  1.000000  0.151734  0.634513  0.427036  0.501175  0.609783  0.610853   
1  0.151734  1.000000  0.146700  0.090673  0.112661  0.132255  0.123194   
2  0.634513  0.146700  1.000000  0.534119  0.647832  0.712363  0.721335   
3  0.427036  0.090673  0.534119  1.000000  0.442705  0.503872  0.508007   
4  0.501175  0.112661  0.647832  0.442705  1.000000  0.562170  0.569140   
5  0.609783  0.132255  0.712363  0.503872  0.562170  1.000000  0.716232   
6  0.610853  0.123194  0.721335  0.508007  0.569140  0.716232  1.000000   
7  0.708673  0.136029  0.755959  0.504530  0.580322  0.634723  0.663807   
8  0.405735  0.123700  0.364653  0.218353  0.264411  0.329350  0.329392   
9  0.477501  0.113725  0.566219  0.384210  0.466017  0.557778  0.547208   

          7         8         9  
0  0.708673  0.405735  0.477501  
1  0.136029  0.123700  0.113725  
2  0.755959  0.364653  0.566219  
3  0.504530  0.218353  0.384210  
4  0.580322  0.264411  0.466017  
5  0.634723  0.329350  0.557778  
6  0.663807  0.329392  0.547208  
7  1.000000  0.316111  0.508498  
8  0.316111  1.000000  0.287614  
9  0.508498  0.287614  1.000000

In [8]:
# create a list of 10 random integers

#importing random module
import random

# the total number of documents
total_docs = len(cleaned_documents)

# generating 10 random unique indices
random_indices = random.sample(range(total_docs), 10)

print("Random document indices:", random_indices)

Random document indices: [69, 639, 393, 21, 369, 552, 215, 360, 350, 456]


In [9]:
# Compute similarity scores for 10 random documents

# getting TF-IDF vectors for the selected random documents
random_tfidf = tfidf_matrix[random_indices]

# computing similarity matrix
random_similarity_matrix = cosine_similarity(random_tfidf)

# finally, displaying as a DataFrame for better viewing
random_similarity_df = pd.DataFrame(random_similarity_matrix)
random_similarity_df

0         1         2         3         4         5         6  \
0  1.000000  0.028517  0.046999  0.199117  0.041066  0.059482  0.027973   
1  0.028517  1.000000  0.019407  0.051257  0.020208  0.026803  0.013076   
2  0.046999  0.019407  1.000000  0.131012  0.032637  0.049922  0.057218   
3  0.199117  0.051257  0.131012  1.000000  0.073075  0.145005  0.060978   
4  0.041066  0.020208  0.032637  0.073075  1.000000  0.052631  0.016317   
5  0.059482  0.026803  0.049922  0.145005  0.052631  1.000000  0.031093   
6  0.027973  0.013076  0.057218  0.060978  0.016317  0.031093  1.000000   
7  0.034019  0.022640  0.027362  0.071735  0.025188  0.033622  0.023797   
8  0.048853  0.022520  0.041391  0.087720  0.037680  0.048669  0.024368   
9  0.043334  0.026667  0.055333  0.117020  0.033493  0.055474  0.022420   

          7         8         9  
0  0.034019  0.048853  0.043334  
1  0.022640  0.022520  0.026667  
2  0.027362  0.041391  0.055333  
3  0.071735  0.087720  0.117020  
4  0.025188  0.037680  0.033493  
5  0.033622  0.048669  0.055474  
6  0.023797  0.024368  0.022420  
7  1.000000  0.027882  0.030928  
8  0.027882  1.000000  0.036649  
9  0.030928  0.036649  1.000000

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [10]:
# Process files and generate chunks

# Okay, let's start by initialising text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        
    chunk_overlap=50,      
    separators=["\n\n", "\n", ".", " "]  
)

# now, splitting documents into chunks
chunked_documents = text_splitter.split_documents(cleaned_documents)

# checking the number of chunks generated
print(f"Original documents: {len(cleaned_documents)}")
print(f"Total chunks generated: {len(chunked_documents)}")

# finally, let's preview a chunk
chunked_documents[0]

Original documents: 695
Total chunks generated: 139240


Document(metadata={'source': 'corpus\\contractnli\\01_Bosch-Automotive-Service-Solutions-Mutual-Non-Disclosure-Agreement-7-12-17.txt'}, page_content='mutual non disclosure agreement subject matter effective date agreement period 2017 exchange information 2017 period confidentiality agreement made effective date agreement noted parties. background i. parties desire discussions relating subject matter purposes evaluating possible business relationship purpose . parties may extend subject matter add additional parties executing one addenda agreement. ii')

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [11]:
# Fetch your OPENAI API Key as an environment variable

os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxx"  #can't share my OpenAI API key here. Cannot commit this to GitHub.

In [12]:
# Initialise an embedding function

embedding_model = OpenAIEmbeddings()

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [13]:
# Add Chunks to vector DB

persist_directory = 'chroma_vector_db' 

# storing chunks and embeddings
vectordb = Chroma.from_documents(
    documents=chunked_documents,           
    embedding=embedding_model,             
    persist_directory=persist_directory    
)

# persisting to disk
vectordb.persist()

# testing
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

print(f"Vector DB loaded with {vectordb._collection.count()} chunks.")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [14]:
# Create a RAG chain

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# initialising LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  
)

NameError: name 'vectordb' is not defined

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [15]:
# Create a function for question answering

def answer_query(query):
    result = rag_chain.invoke({"query": query})
    print("Answer:")
    print(result["result"])
    print("\nSources:")
    for doc in result["source_documents"]:
        print(doc.metadata.get("source", "N/A"))

In [16]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"

question = "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"

answer_query(question)

NameError: name 'rag_chain' is not defined

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [18]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set

benchmark_files = glob.glob('benchmark/*.json')

all_questions = []
all_ground_truths = []

# looping through each JSON file
for file_path in benchmark_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data:
            question = item.get("question", "").strip()
            answer = item.get("ground_truth", "").strip() or item.get("answer", "").strip()
            if question and answer:
                all_questions.append(question)
                all_ground_truths.append(answer)

#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [22]:
# Function to evaluate the RAG pipeline

# loading BLEU scorer
bleu = load_metric("bleu")

# initialising ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_rag_pipeline(questions, ground_truths, qa_function, max_q=50):
    results = []

    for i in tqdm(range(min(len(questions), max_q))):  # limit to 50 for speed
        q = questions[i]
        true_ans = ground_truths[i]
        
        try:
            pred = qa_function(q)["result"]
        except:
            pred = ""
        
        # computing BLEU
        bleu_score = bleu.compute(predictions=[pred.split()], references=[[true_ans.split()]])['bleu']
        
        # computing ROUGE
        rouge_scores = rouge.score(true_ans, pred)
        
        results.append({
            "question": q,
            "ground_truth": true_ans,
            "predicted": pred,
            "bleu": bleu_score,
            "rouge1": rouge_scores['rouge1'].fmeasure,
            "rouge2": rouge_scores['rouge2'].fmeasure,
            "rougeL": rouge_scores['rougeL'].fmeasure,
        })
    
    return pd.DataFrame(results)

# defining QA function

def qa_function(q):
    return rag_chain.invoke({"query": q})

# running evaluation

eval_df = evaluate_rag_pipeline(all_questions, all_ground_truths, qa_function, max_q=50)
eval_df.head()

0it [00:00, ?it/s]


Empty DataFrame
Columns: []
Index: []

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [23]:
# Evaluate the RAG pipeline


# evaluating 1st 100 questions
eval_df = evaluate_rag_pipeline(all_questions, all_ground_truths, qa_function, max_q=100)

# show top rows
eval_df.head()

0it [00:00, ?it/s]


Empty DataFrame
Columns: []
Index: []

## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

## Data Understanding and Preparation

The corpus consisted of four categories of legal documents: non-disclosure agreements, merger/acquisition contracts, clause-annotated contracts, and privacy policies. <br>These were read from .txt files, cleaned to remove noise (e.g., email addresses, phone numbers, special characters), and normalised for analysis. <br>The documents were then chunked using a recursive character-based splitter to preserve legal clause integrity while adhering to model token limits.

## EDA

An analysis of document lengths showed significant variation, highlighting the complexity and inconsistency inherent in legal contracts. <br>Word frequency analysis surfaced both common legal terminology and low-frequency, domain-specific phrases. <br>TF-IDF-based similarity matrices helped identify structurally similar documents and clause patterns, which could be useful for clustering or de-duplication in future applications.

## RAG Pipeline Design & Functionality

Using OpenAI embeddings and ChromaDB as the vector store, a retrieval component was implemented to fetch the top-k relevant chunks for each question. <br>The retrieval results were passed to a GPT-3.5 model via a LangChain RetrievalQA wrapper to generate responses. <br>This pipeline allowed dynamic, context-aware question answering without fine-tuning on legal data.



## Evaluation & Observtions

The model was evaluated on a benchmark set of 100 legal QA pairs. Performance was assessed using BLEU and ROUGE metrics.<br>The average ROUGE-1 score indicated moderate overlap between generated answers and reference answers, particularly for fact-based questions.<br>BLEU scores were comparatively lower, as expected, due to the generative flexibility of large language models.

The RAG system performed well when the question closely matched the language and structure found in the corpus. However, performance declined on ambiguous or multi-step reasoning questions, likely due to limitations in chunking and retrieval granularity.
The retrieval quality heavily influences the overall performance of a RAG pipeline. 

The retrieval quality heavily influences the overall performance of a RAG pipeline. Improving chunking strategies (e.g., clause-aware splitting) or incorporating hybrid retrieval (dense + keyword) could improve results.

The current setup demonstrated how legal document analysis can be accelerated using retrieval-based LLM pipelines without domain-specific model tuning.

Future iterations could benefit from improved reranking mechanisms, better prompt engineering, and user feedback loops to refine retrieval accuracy and answer relevance.